# Async/Await Examples for Deadline 2

This notebook demonstrates **meaningful** vs **trivial** usage of async/await patterns in C# applications.

## 🎯 **Learning Objectives**
- Understand when async/await provides real benefits
- Learn proper async patterns and best practices
- Avoid common async pitfalls and anti-patterns
- Implement async operations that improve application responsiveness

## ✅ **Meaningful Async/Await Usage Examples**

### 1. Database Operations (I/O Bound)

In [ ]:
// ✅ MEANINGFUL: Database operations are I/O bound and benefit from async
public class UserService
{
    private readonly DbContext _context;
    
    public async Task<User> GetUserByIdAsync(int id)
    {
        // This releases the thread while waiting for database I/O
        return await _context.Users
            .Where(u => u.Id == id)
            .FirstOrDefaultAsync();
    }
    
    public async Task<List<User>> GetActiveUsersAsync()
    {
        // Complex query that benefits from async execution
        return await _context.Users
            .Where(u => u.IsActive && u.LastLoginDate > DateTime.Now.AddDays(-30))
            .Include(u => u.Profile)
            .ToListAsync();
    }
    
    // ✅ GOOD: Proper async propagation with cancellation support
    public async Task<bool> UpdateUserAsync(User user, CancellationToken cancellationToken = default)
    {
        try
        {
            _context.Users.Update(user);
            var result = await _context.SaveChangesAsync(cancellationToken);
            return result > 0;
        }
        catch (OperationCanceledException)
        {
            // Handle cancellation gracefully
            return false;
        }
    }
}

### 2. HTTP Client Operations (Network I/O)

In [ ]:
// ✅ MEANINGFUL: HTTP calls are network I/O operations that benefit from async
public class ExternalApiService
{
    private readonly HttpClient _httpClient;
    private readonly ILogger<ExternalApiService> _logger;
    
    public async Task<ApiResponse<T>> GetDataAsync<T>(string endpoint, CancellationToken cancellationToken = default)
    {
        try
        {
            using var response = await _httpClient.GetAsync(endpoint, cancellationToken);
            
            if (response.IsSuccessStatusCode)
            {
                var content = await response.Content.ReadAsStringAsync();
                var data = JsonSerializer.Deserialize<T>(content);
                return new ApiResponse<T> { Data = data, IsSuccess = true };
            }
            
            return new ApiResponse<T> { IsSuccess = false, ErrorMessage = response.ReasonPhrase };
        }
        catch (HttpRequestException ex)
        {
            _logger.LogError(ex, "HTTP request failed for endpoint: {Endpoint}", endpoint);
            return new ApiResponse<T> { IsSuccess = false, ErrorMessage = ex.Message };
        }
        catch (TaskCanceledException ex) when (ex.InnerException is TimeoutException)
        {
            _logger.LogWarning("Request timeout for endpoint: {Endpoint}", endpoint);
            return new ApiResponse<T> { IsSuccess = false, ErrorMessage = "Request timed out" };
        }
    }
    
    // ✅ GOOD: Parallel async operations for better performance
    public async Task<Dictionary<string, object>> GetMultipleDataSourcesAsync()
    {
        var userTask = GetDataAsync<User[]>("users");
        var ordersTask = GetDataAsync<Order[]>("orders");
        var productsTask = GetDataAsync<Product[]>("products");
        
        // All requests run in parallel, wait for all to complete
        await Task.WhenAll(userTask, ordersTask, productsTask);
        
        return new Dictionary<string, object>
        {
            ["users"] = await userTask,
            ["orders"] = await ordersTask,
            ["products"] = await productsTask
        };
    }
}

### 3. File Operations (I/O Bound)

In [ ]:
// ✅ MEANINGFUL: File I/O operations benefit from async, especially for large files
public class FileProcessingService
{
    public async Task<string> ReadLargeFileAsync(string filePath)
    {
        // Reading large files asynchronously prevents UI blocking
        using var reader = new StreamReader(filePath);
        return await reader.ReadToEndAsync();
    }
    
    public async Task ProcessCsvFileAsync(string filePath, IProgress<int> progress = null)
    {
        using var reader = new StreamReader(filePath);
        var lineCount = 0;
        var processedCount = 0;
        
        // Count total lines first (could be improved with file size estimation)
        var content = await reader.ReadToEndAsync();
        lineCount = content.Split('\n').Length;
        
        // Reset stream position
        reader.BaseStream.Position = 0;
        reader.DiscardBufferedData();
        
        string line;
        while ((line = await reader.ReadLineAsync()) != null)
        {
            // Process each line asynchronously
            await ProcessLineAsync(line);
            processedCount++;
            
            // Report progress periodically
            if (processedCount % 100 == 0)
            {
                progress?.Report((int)((double)processedCount / lineCount * 100));
            }
        }
    }
    
    private async Task ProcessLineAsync(string line)
    {
        // Simulate processing that might involve async operations
        var data = line.Split(',');
        if (data.Length > 0 && !string.IsNullOrWhiteSpace(data[0]))
        {
            // This could be a database call, API call, etc.
            await Task.Delay(1); // Placeholder for real async work
        }
    }
}

## ❌ **Trivial/Incorrect Async/Await Usage Examples**

### Anti-patterns to avoid:

In [ ]:
// ❌ TRIVIAL: Adding async to CPU-bound operations provides no benefit
public async Task<int> AddNumbersAsync(int a, int b)
{
    // This is just CPU work, async adds overhead without benefit
    return await Task.FromResult(a + b); // Don't do this!
}

// ❌ BLOCKING: Blocking on async operations defeats the purpose
public string GetDataSync()
{
    // This blocks the thread and can cause deadlocks
    return GetDataAsync().Result; // Don't do this!
}

// ❌ FIRE AND FORGET: Not awaiting async operations
public void ProcessDataBadly()
{
    // This starts the operation but doesn't wait for completion
    ProcessDataAsync(); // Don't do this!
    // Code continues without knowing if the operation succeeded
}

// ❌ UNNECESSARY ASYNC: Wrapping synchronous operations
public async Task<User> GetUserBadlyAsync(int id)
{
    // If the underlying operation is sync, don't make it async
    return await Task.FromResult(new User { Id = id }); // Don't do this!
}

// ❌ DEADLOCK PRONE: Blocking in async context
public async Task<string> DeadlockRiskAsync()
{
    var task = SomeAsyncOperation();
    // This can cause deadlocks in certain contexts (like ASP.NET with sync context)
    return task.Result; // Don't do this!
}

## 🔧 **Best Practices for Async/Await**

### 1. Proper Exception Handling

### 🚫 Bad Example: Poor Exception Handling in Async

The following code demonstrates a common mistake: not handling exceptions in async methods, which can lead to unobserved task exceptions and unpredictable failures.

In [ ]:
// 🚫 BAD: No exception handling in async method
public async Task<string> GetDataAsync()
{
    // If GetDataFromApiAsync throws, the exception is unobserved until awaited
    var data = await GetDataFromApiAsync();
    // If an exception occurs above, it will propagate up and may crash the app
    return data;
}

// 🚫 BAD: Swallowing exceptions without logging or handling
public async Task<string> GetDataWithSilentFailAsync()
{
    try
    {
        var data = await GetDataFromApiAsync();
        return data;
    }
    catch
    {
        // Silent fail: exception is swallowed, caller gets null or default
        return null;
    }
}

In [ ]:
// ✅ GOOD: Proper async exception handling
public async Task<ApiResult<T>> SafeAsyncOperationAsync<T>(Func<Task<T>> operation)
{
    try
    {
        var result = await operation();
        return ApiResult<T>.Success(result);
    }
    catch (OperationCanceledException)
    {
        return ApiResult<T>.Cancelled();
    }
    catch (HttpRequestException ex)
    {
        return ApiResult<T>.Error($"Network error: {ex.Message}");
    }
    catch (Exception ex)
    {
        // Log the exception
        return ApiResult<T>.Error($"Unexpected error: {ex.Message}");
    }
}

### 2. ConfigureAwait Usage

In [ ]:
// ✅ GOOD: Using ConfigureAwait(false) in library code
public async Task<string> LibraryMethodAsync()
{
    // In library code, use ConfigureAwait(false) to avoid deadlocks
    var data = await GetDataFromApiAsync().ConfigureAwait(false);
    var processed = await ProcessDataAsync(data).ConfigureAwait(false);
    return processed;
}

// ✅ GOOD: Don't use ConfigureAwait(false) when you need the context
public async Task UiUpdateMethodAsync()
{
    var data = await GetDataAsync(); // Need UI context for updates
    // Update UI elements here - requires original context
    UpdateUIWithData(data);
}

### 3. Cancellation Token Support

In [ ]:
// ✅ GOOD: Proper cancellation token usage
public async Task<List<ProcessedItem>> ProcessItemsAsync(
    IEnumerable<Item> items, 
    CancellationToken cancellationToken = default)
{
    var results = new List<ProcessedItem>();
    
    foreach (var item in items)
    {
        // Check for cancellation before each iteration
        cancellationToken.ThrowIfCancellationRequested();
        
        try
        {
            // Pass cancellation token to async operations
            var processed = await ProcessSingleItemAsync(item, cancellationToken);
            results.Add(processed);
        }
        catch (OperationCanceledException)
        {
            // Return partial results if cancelled
            break;
        }
    }
    
    return results;
}

private async Task<ProcessedItem> ProcessSingleItemAsync(Item item, CancellationToken cancellationToken)
{
    // Simulate processing with cancellation support
    using var timeout = new CancellationTokenSource(TimeSpan.FromSeconds(30));
    using var combined = CancellationTokenSource.CreateLinkedTokenSource(
        cancellationToken, timeout.Token);
    
    return await DoActualProcessingAsync(item, combined.Token);
}

## 🚀 **Advanced Async Patterns**

### 1. Producer-Consumer with Async

In [ ]:
// ✅ ADVANCED: Async producer-consumer pattern
public class AsyncDataProcessor
{
    private readonly Channel<WorkItem> _channel;
    private readonly ChannelWriter<WorkItem> _writer;
    private readonly ChannelReader<WorkItem> _reader;
    
    public AsyncDataProcessor(int capacity = 100)
    {
        var options = new BoundedChannelOptions(capacity)
        {
            WaitForReaderPolicy = BoundedChannelFullMode.Wait,
            SingleReader = false,
            SingleWriter = false
        };
        
        _channel = Channel.CreateBounded<WorkItem>(options);
        _writer = _channel.Writer;
        _reader = _channel.Reader;
    }
    
    // Producer
    public async Task ProduceWorkAsync(IEnumerable<WorkItem> items, CancellationToken cancellationToken = default)
    {
        try
        {
            foreach (var item in items)
            {
                await _writer.WriteAsync(item, cancellationToken);
            }
        }
        finally
        {
            _writer.Complete();
        }
    }
    
    // Consumer
    public async Task ConsumeWorkAsync(Func<WorkItem, Task> processor, CancellationToken cancellationToken = default)
    {
        await foreach (var item in _reader.ReadAllAsync(cancellationToken))
        {
            try
            {
                await processor(item);
            }
            catch (Exception ex)
            {
                // Log error but continue processing
                Console.WriteLine($"Error processing item {item.Id}: {ex.Message}");
            }
        }
    }
}

## 📝 **Key Takeaways**

### ✅ **Use Async/Await When:**
- Performing I/O operations (database, network, file system)
- You need to keep UI responsive during long operations
- Working with naturally asynchronous APIs
- Coordinating multiple concurrent operations

### ❌ **Don't Use Async/Await When:**
- Performing pure CPU-bound calculations
- The operation is already synchronous and fast
- You would just wrap sync code in Task.FromResult

### 🔧 **Best Practices:**
1. **Consider** CancellationToken for long-running operations
2. **Never** block on async code with .Result or .Wait()
3. **Use** ConfigureAwait(false) in library code
4. **Handle** exceptions properly in async methods
5. **Avoid** async void except for event handlers
6. **Prefer** Task.WhenAll for parallel operations
7. **Consider** using Channels for producer-consumer scenarios